In [44]:
import pandas as pd
from shapely.geometry import Point, Polygon
import numpy as np
import json
from geojson import utils #used to get coordinates from geo json

In [141]:
hk = json.load(open("../../datasets/hk_geo.json"))

polygons  = [ Polygon(utils.coords(each)) for each in hk["features"]] 

for each in hk["features"]:
    each["name"] = each["properties"]["NAME_1"]
    del each["properties"]

districts = [each["name"] for each in hk["features"]]

In [142]:
def find_district(point,polygons=polygons,districts=districts):
    
    assert len(districts) == len(polygons)
    
    for i,polygon in enumerate(polygons):
        if polygon.contains(point):
            return districts[i]
    return np.nan


In [ ]:
Po

# Hospitals

In [146]:
general = pd.read_csv("../../datasets/General_Outpatient_Clinics.csv")
general["district"] = general.apply(lambda row: find_district(Point(row["Long"],row["Lat"])),axis=1)
general["type"] = "general"

In [152]:
private = pd.read_csv("../../datasets/Private_Clinics.csv")
private["district"] = private.apply(lambda row: find_district(Point(row["Long"],row["Lat"])),axis=1)
private["type"] = "private"

private = private.rename(columns={"Name of Clinic":"Name"})

In [153]:
hospitals = pd.concat([general,private])

/home/dom/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [154]:
hospitals = hospitals.drop(columns=["Unnamed: 0"])
hospitals.reset_index(inplace=True)

In [160]:
hospitals = hospitals[~hospitals.district.isna()]

In [168]:
hospitals["x"] = 1

In [172]:
hospital_counts = hospitals.groupby(hospitals.district).x.sum().reset_index()

In [175]:
hospital_counts = hospital_counts.rename(columns={"district":"name","x":"count"})

In [176]:
hospital_counts.to_json("../../datasets/hospitals.json",orient="records")

In [177]:
hospital_counts

,name,count
0,Central and Western,9
1,Eastern,10
2,Islands,4
3,Kowloon City,14
4,Kwai Tsing,7
5,Kwun Tong,13
6,North,5
7,Sai Kung,7
8,Sha Tin,8
9,Sham Shui Po,6


# Food

In [27]:
# pd.read_excel("../../datasets/Hong Kong Food Licences - Final.xlsx")